# Deploy a simple ML pipeline for expense report estimation

## Introduction
In the prior tutorial, we saw how to train and register a model using Azure Machine Learning pipelines.  This tutorial shows how we can deploy the model to Azure Machine Learning.

We will first deploy the results locally so that we can ensure that deployment succeeds.  Troubleshooting deployed models using only Azure Machine Learning's Studio is **very** difficult, but it can be a lot easier if you try to deploy locally, as then you can see the Docker logs and see what is happening.

In [ ]:
from azureml.core.webservice import Webservice, LocalWebservice, AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

## Deploying the Model

Our first step is to load the workspace where we deployed our model.  Then, we will load that model into the `model` object.

In [ ]:
ws = Workspace.from_config()
model = Model(ws, 'ExpenseReportsPipelineModel')

Just as with model training, you have a few options for deciding on the environment you'd like to use.  In this case, we will once again use the training environment.  There is a separate inference environment which has less installed, but the downside to using the inference-only environment is that it does not have the Azure ML SDK pre-installed, so we'll go with what is technically a training image.

In [ ]:
env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=5)

The `inference_config` object tells the REST API what, exactly, the specification should look like for the scoring endpoint.

### Deploying the Model Locally

The `deployment_config` here is a local web service which will run on port 6789.  Note that this does require that you have Docker installed and set up on your machine.

In [ ]:
inference_config = InferenceConfig(entry_script="src/score.py", environment=env)
deployment_config = LocalWebservice.deploy_configuration(port=6789)


This code deploys our pipeline-created expense reports model to our local deployment configuration.  The name of the deployment is `expense-reports-score-local` and we will be able to use it to perform local testing.

In [ ]:
local_service = Model.deploy(workspace=ws, 
                       name='expense-reports-score-local', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config = deployment_config)

Wait for the deployment to complete and then paste out the URI that you'd need to call in order to score new expense reports.

In [ ]:
local_service.wait_for_deployment(show_output=True)
print(f"Scoring URI is : {local_service.scoring_uri}")

### Deploying the Model via Azure Container Instance (ACI)

Now that we've tested that the model works, we can deploy it via Azure Container Instances (ACI).  Note that Azure ML also supports deployment to Azure Kubernetes Service (AKS), and that AKS is much more production-ready than ACI.  You can safely use ACI for development and testing scenarios, but for production, Microsoft highly recommends AKS.

In this case, we will declare that we want 1 CPU core and 2 GB of memory.  These are overkill for the service in question, but note that they are configurable based on your specific needs.

In [ ]:
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

The deployment process looks almost exactly the same as above.  The only differences are the name of the deployment and the new deployment config.  This is nice because it intimates that you can use the same code to deploy different types of models to different locations, including even hosting on different platforms between development and production.

In [ ]:
aci_service = Model.deploy(workspace=ws, 
                       name='expense-reports-score-aci', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config = aci_deployment_config)

Once we've wrapped everything up, print out the scoring URI.  This can take 5-10 minutes, so be patient at this step.

In [ ]:
aci_service.wait_for_deployment(show_output=True)
print(f"Scoring URI is : {aci_service.scoring_uri}")